In [104]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import csv
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types


# for named entity
from nltk import word_tokenize, pos_tag, ne_chunk, tree, download
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# for preparing data
download('vader_lexicon')
%matplotlib inline


# nltk for natural language processing

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/chitrankdixit/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [105]:
path =r'./playstorereviewsdata/' # use your path
allFiles = glob.glob(path + "/*.tsv")

# for out_count, file in enumerate(allFiles):
#     with open(file, newline='') as csvfile:
#         spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
#         #import pdb;pdb.set_trace()
#         for counter, row in enumerate(spamreader):
#             import pdb;pdb.set_trace()
#             print(row)
            
#     with open(file+out_count, 'wt') as csvfile:
#             writer = csv.writer(csvfile, delimiter='', quotechar='|')
#             #writer.writerow(["#"] + anarkali_characteristics)
#             #import pdb;pdb.set_trace()
#             for element in lehenga_characteristics:
#                 #import pdb;pdb.set_trace()
#                 writer.writerow([element["name"], element["count"]])

In [106]:
# get full data


frame = pd.DataFrame()
list_ = []

# datacolumns = [
#     "Package Name",
#     "App Version Code",
#     "Reviewer Language",
#     "Device",
#     "Review Submit Date and Time",
#     "Review Submit Millis Since Epoch",
#     "Review Last Update Date and Time",
#     "Review Last Update Millis Since Epoch",
#     "Star Rating",
#     "Review Title",
#     "Review Text",
#     "Developer Reply Date and Time",
#     "Developer Reply Millis Since Epoch",
#     "Developer Reply Text,Review Link"
# ]

for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, encoding = "ISO-8859-1", error_bad_lines=False, sep='\t')
    list_.append(df)
    
    
full_data = pd.concat(list_)
full_data.head(5)

mail_data = pd.read_csv("parsed_data.tsv",index_col=None, header=0, encoding = "ISO-8859-1", error_bad_lines=False, sep='\t')

In [107]:
# remove the NaN 
full_data["Review Text"] = full_data["Review Text"].fillna('')
full_data["Review Title"] = full_data["Review Title"].fillna('')
full_data["App Version Code"] = full_data["App Version Code"].fillna(0.0)
full_data["App Version Name"] = full_data["App Version Name"].fillna('')
full_data["Developer Reply Date and Time"] = full_data["Developer Reply Date and Time"].fillna('')
full_data["Developer Reply Millis Since Epoch"] = full_data["Developer Reply Millis Since Epoch"].fillna('')
full_data["Developer Reply Text"] = full_data["Developer Reply Text"].fillna('')

# slicing the required columns only
full_data = full_data[["Review Text", "Review Title"]]

In [108]:
# clean mail data
mail_data

,requester_name,subject,ticket_id,description
0,Rachel,This is a sample ticket,9000639244,"This is a sample ticket, feel free to delete it."
1,Rachel,Woohoo.. Your Freshdesk Test Mail,9000639285,Cool! You've received the test mail yo...
2,Sourav Poddar,Test email - please check my order status,9000639357,"Hi, Please check my order status. Thanks."
3,Flyrobe Team,(#616150162) Flyrobe Forwarding Confirmation -...,9000640161,sourav.poddar@flyrobe.com has requested to aut...
4,Sourav Poddar,Testing orders,9000640362,Please blah blah
5,shreya guchait,NaN,9000642990,Hy I want to rent a dress from 31st to 8 Jan w...
6,Rituchugh97,Re:,9000643017,"Sorry, please do cancel this order. Sent fro..."
7,Zunera Zari,Issue with return,9000648667,"Hi team, I was sleeping full day so dint answ..."
8,apurva Jain,Confirming Order,9000656683,"Hi Flyrobe Team, Thank you for sending Lehan..."
9,Nikita Kanchan,Fwd: Dress details,9000660319,"Hi, I had a call with one of your representat..."


In [109]:
full_data.columns = ["Description","Subject"]
full_data.head()

,Description,Subject
0,This is a brilliant concept. Please start for ...,Great product!
1,Brilliant concept :),
2,,
3,Brilliant stuff!! Waiting to see more..,
4,,


In [110]:
mail_data = mail_data[["subject", "description"]]
mail_data.columns = ["Subject", "Description"]
mail_data.head()

# when mail data get cleaned properly then use the below
#full_data = full_data.append(mail_data)

,Subject,Description
0,This is a sample ticket,"This is a sample ticket, feel free to delete it."
1,Woohoo.. Your Freshdesk Test Mail,Cool! You've received the test mail yo...
2,Test email - please check my order status,"Hi, Please check my order status. Thanks."
3,(#616150162) Flyrobe Forwarding Confirmation -...,sourav.poddar@flyrobe.com has requested to aut...
4,Testing orders,Please blah blah


In [111]:
full_data.tail()

,Description,Subject
43,the customer service of the jabong shopping ap...,
44,Good collection.,
45,,
46,,
47,,


In [112]:
full_data = full_data.loc[full_data['Description'] != '']



full_data.iloc[1078]['Description']

'Job application provides a lot of great experiences I am definitely going to use it application again and again'

In [113]:
sid = SentimentIntensityAnalyzer()
client = language.LanguageServiceClient()

# add a named entities to look for
#import pdb;pdb.set_trace()
negation_list = ["No", "Not"]
def get_sentiment_score(item):
    sentences = item['Description']#.encode('utf-16')
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects the sentiment of the text
    sentiment = client.analyze_sentiment(document=document).document_sentiment

#     negate = None
#     for element in negation_list:
#         m = re.findall('^'+element+' \w+', sentences)
#         if m:
#             break
#     if m:
#         negate, sentences = sentences[:2],sentences[2:]
#     ss = sid.polarity_scores(sentences)
#     if negate:
#         if ss['neg'] > max(ss['pos'], ss['neu']): 
#             ss['pos'] = 1.0
#             ss['neu'] = 0.0
#             ss['neg'] = 0.0
#         elif ss['pos'] > max(ss['neg'], ss['neu']): 
#             ss['pos'] = 0.0
#             ss['neu'] = 0.0
#             ss['neg'] = 1.0
# #         elif ss['neu'] > max(ss['pos'], ss['neg']):
# #             ss['pos'] = 1.0
# #             ss['neu'] = 0.0
# #             ss['neg'] = 0.0
    
#     negative = ss["neg"]
#     positive = ss["pos"]
#     neutral = ss["neu"]
    #compound = ss["compound"]
    return pd.Series({'negative': negative, 'positive': positive, 'neutral': neutral})


scores = full_data.apply(get_sentiment_score, axis=1)

final_data = pd.concat([ full_data , scores], axis=1)

In [114]:
def get_sentiment_class(item):
    sentiment_class = 0
    score_list = [item['neutral'], item['negative'], item['positive']]
    max_val = max(score_list)
    
    sentiment_class = score_list.index(max_val)
    
    return pd.Series({'sentiment_class': sentiment_class})


score_class = final_data.apply(get_sentiment_class, axis=1)

final_sliced_data = pd.concat([ final_data , score_class], axis=1)


In [115]:
# bring the data with probability tech + , tech - , ops + , ops - , product + , product -
import re
tech_list = ["order", "placed", "tech", "issue", "coupon", "item", "order placed", "coupon issue", "tech issue", "App crash", "web crash", "ios", 
            "android", "signup issue", "signin issue", "order placed", "order confirmation", "online payment", "hang", "too slow"
            , "cart", "image not matching"]
ops_list = ["delivery", "measurement", "pickup", "refund",  "fitting", "trial", "pickup", "refund", "deposit", "on-time", "delay", "address updation", "return", "cancellation", "exchange",
"payment issues"]
product_list = ["content", "available", "error", "loading", "website not loading", "site not loading", "content not available", "cloth tear", "defective product", "discount codes"]

def calculate_token_probability(item):
    tech = 0; ops = 0; product = 0
    tokens = item['Description'].split(' ')
#     for element in tokens:
#         #import pdb;pdb.set_trace()
#         try:
#             tech_match = re.findall(element.lower(), tech_str)
#         except Exception:
#             tech_match = None
            
#         try:
#             ops_match = re.findall(element.lower(), ops_str)
#         except Exception:
#             ops_match = None
            
#         try:
#             product_match = re.findall(element.lower(), product_str)
#         except Exception:
#             product_match = None
#         if tech_match:
#             tech += 1
#         elif ops_match:
#             ops += 1
#         elif product_match:
#             product += 1
    tech = [1 for tech_i in tech_list if tech_i.lower() in item['Description']]
    ops = [1 for ops_i in ops_list if ops_i.lower() in item['Description']]
    product = [1 for product_i in product_list if product_i.lower() in item['Description']]
    

#     for tech_i in tech_list:
#         try:
#             tech_match = re.findall(tech_i.lower(), item['Review Text'])
#         except Exception:
#             tech_match = None
            
#         if tech_match:
#              tech += 1
    
#     for ops_i in ops_list:
#         try:
#             ops_match = re.findall(ops_i.lower(), item['Review Text'])
#         except Exception:
#             ops_match = None
            
#         if ops_match:
#              ops += 1
    
#     for product_i in product_list:
#         try:
#             product_match = re.findall(product_i.lower(), item['Review Text'])
#         except Exception:
#             product_match = None
            
#         if product_match:
#              product += 1
            
    tech_sum = sum(tech)
    ops_sum = sum(ops)
    product_sum = sum(product)
    
    sum_all = tech_sum + ops_sum + product_sum
    try:
        return pd.Series({'tech_prob': tech_sum/sum_all, 'ops_prob': ops_sum/sum_all, 'product_prob': product_sum/sum_all})
    except Exception: 
        return pd.Series({'tech_prob': 0, 'ops_prob': 0, 'product_prob': 0})
    
    
# def cal(item):
#     print(item['Review Text'])
#     return pd.Series({'hello': item}, )
    
# probability = sliced_data.apply(calculate_token_probability, axis=1)

# final_sliced_data = pd.concat([ sliced_data , probability], axis=1)


# final_sliced_data['ops_positive'] = (( final_sliced_data['ops_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Positive"] == True).astype(int).values
# final_sliced_data['product_positive'] = (( final_sliced_data['product_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Positive"] == True ).astype(int).values
# final_sliced_data['tech_positive'] = (( final_sliced_data['tech_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1)  ) & final_sliced_data["Positive"] == True).astype(int).values
# final_sliced_data['ops_negative'] = (( final_sliced_data['ops_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Negative"] == True).astype(int).values
# final_sliced_data['product_negative'] = (( final_sliced_data['product_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Negative"] == True).astype(int).values
# final_sliced_data['tech_negative'] = (( final_sliced_data['tech_prob'] == final_sliced_data[['ops_prob', 'tech_prob', 'product_prob']].max(axis=1) ) & final_sliced_data["Negative"] == True).astype(int).values
# final_sliced_data

In [116]:
final_sliced_data['Description'].head(5)

0    This is a brilliant concept. Please start for ...
1                                 Brilliant concept :)
3              Brilliant stuff!! Waiting to see more..
5    This idea will be considered as one of the top...
7    No spending on buying apparels for gf... Why t...
Name: Description, dtype: object

In [117]:
final_sliced_data.head(5)

,Description,Subject,negative,neutral,positive,sentiment_class
0,This is a brilliant concept. Please start for ...,Great product!,0.0,0.534,0.466,0
1,Brilliant concept :),,0.0,0.128,0.872,2
3,Brilliant stuff!! Waiting to see more..,,0.0,0.533,0.467,0
5,This idea will be considered as one of the top...,Excellent,0.0,0.715,0.285,0
7,No spending on buying apparels for gf... Why t...,Brilliant ..this is what we needed!!,0.0,1.000,0.000,0


In [118]:
# generate csv file
final_sliced_data.to_csv("text_data.tsv", sep='\t')

In [119]:
final_sliced_data.to_csv('pos_neg_score.csv', index = False)

In [120]:
print(final_sliced_data.loc[final_sliced_data['sentiment_class'] == 0].__len__())
print(final_sliced_data.loc[final_sliced_data['sentiment_class'] == 1].__len__())
print(final_sliced_data.loc[final_sliced_data['sentiment_class'] == 2].__len__())

7999
30
640


In [121]:
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy


def format_sentence(sent):
    return({word: True for word in word_tokenize(sent)})

In [122]:
labels = final_sliced_data["sentiment_class"].values;

#import pdb;pdb.set_trace()  
kf = StratifiedKFold(n_splits=4)


totalNBV = 0

totalMatNBV = np.zeros((3,3));

# tr, te = kf.split(sliced_data['Review Text'], labels)
# td = []
# for i in train_index:
#     td.append(sliced_data['Review Text'][i])
#import pdb;pdb.set_trace()
review_text = final_sliced_data['Description'].tolist()

# break words
review_words = []

# for item in review_text:
#     review_words += item.split(" ")
counter = 0
for train_index, test_index in kf.split(review_text, labels):
    #import pdb;pdb.set_trace()
    #print(train_index.__len__(), test_index.__len__())
    # X_train = [review_text[i] for i in train_index]
    # X_test = [review_text[i] for i in test_index]
    #import pdb;pdb.set_trace()
    X_train = [[format_sentence(review_text[i]), int(final_sliced_data.iloc[i]['sentiment_class'])] for i in train_index]
    X_test = [[format_sentence(review_text[i]), int(final_sliced_data.iloc[i]['sentiment_class'])] for i in test_index]
    
    X_train_nb = [[list(map(int, format_sentence(review_text[i]).values())), int(final_sliced_data.iloc[i]['sentiment_class'])] for i in train_index]
    X_test_nb = [[list(map(int, format_sentence(review_text[i]).values())), int(final_sliced_data.iloc[i]['sentiment_class'])] for i in test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    
    #vectorizer = TfidfVectorizer(min_df=5, max_df = 0.8, sublinear_tf=True, use_idf=False, stop_words='english')
    #train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
    #test_corpus_tf_idf = vectorizer.transform(X_test)
    nb = MultinomialNB()#, GaussianNB
    #import pdb;pdb.set_trace()
    nb.fit(X_train_nb, y_train)
    nb.predict(X_test_nb)
    nbc = NaiveBayesClassifier.train(X_train)
    print(accuracy(nbc, X_test))
    print(accuracy(nb, X_test_nb))
    
    example1 = "Cats are awesome!"
 
    print(nbc.classify(format_sentence(example1)))
    
    example2 = "I don’t like cats."
 
    print(nbc.classify(format_sentence(example2)))
    result1 = []
    for item in X_test:
        result1.append(nbc.classify(item[0]))

ValueError: setting an array element with a sequence.

In [ ]:
ss = sid.polarity_scores(sentences)